In [1]:
import pandas as pd
import numpy as np
import altair as alt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer

np.random.seed(1234)

# Introduction
In this project, we aim to use a KNearestNeighbors classifier model to determine which parameters are most useful in heart disease predicting occurrences using a dataset from UC Irvine (Janosi et al. 1988). Our main goal is understanding how different medical attributes relate to cardiac health outcomes and ultimately creating a reliable predictive model. We aim to uncover meaningful patterns and correlations by carefully exploring and analyzing the dataset. This will enable us to develop a practical framework for estimating the likelihood of heart disease based on an individual's medical profile. Our project aims to answer the following question: What are the most relevant parameters in predicting cardiovascular health?  

# Data Description
Our chosen  dataset comprises 76 attributes, yet published experiments typically utilize a subset of 14 attributes. Among these, the Cleveland database stands out as the primary focus for machine learning researchers. The "goal" field denotes the presence of heart disease in patients, with integer values ranging from 0 (absence) to 4 (Janosi et al. 1988). Previous studies with the Cleveland database have primarily aimed to differentiate between the presence (values 1, 2, 3, 4) and absence (value 0) of heart disease (Janosi et al. 1988).

The features included in the data are: age, sex, constrictive pericarditis (cp), resting blood pressure (trestbps), fasting blood sugar (fbs), resting electrocardiographic measure (restecg), Maximum heart rates (thalach), exercise induced angina (exang),  slope (up, flat, down) %  number of vessels colored (Oldpeak), st segment/heart rate slope (Slope), class (num), number of major vessels (ca), and inherited blood disorder that causes less hemoglobin levels (thal) (Janosi et al. 1988). 

Below, we have provided the reasoning for why we chose to include each predictor value:   

**Age**: basic demographic information, also age is relevant to increased risk of many diseases

**Sex**: basic demographic information, also there may be differences of heart disease diagnosis due to physical differences between sex

**Constrictive Pericarditis (cp)**: a condition where ventricular filling is restricted due to loss of pericardial elasticity, this condition could appear as a form of heart failure

**Resting Blood Pressure (Trestbps)**: blood pressure is related to heart disease since high blood pressure means that the heart is required to put more force to get blood running.

**Fasting Blood Sugar (fbs)**: diabetes is a major risk factor for heart failure

**resting electrocardiographic measure (restecg)**: this indicates if the individual has abnormality of ST-T wave(predicts for morbidity of cardiovascular disease) , left ventricular hypertrophy (increased pressure in heart), or normal

**Maximum heart rates (thalach)**: Provides insight into the cardiovascular health and capacity 

**Exercise-induced angina(Exang)**:Chest pain/discomfort from decreased oxygenated blood at heart muscles often seen in those with cardiovascular disease

**Ulope (up, flat, down) %  number of vessels colored (Oldpeak)**: ST depression induced by exercise relative to rest to allow insight into cardiovascular health 

**st segment/heart rate slope (Slope)**: electrocardiographic criteria for diagnosing accurate coronary artery disease

**Class (Num)**: Used for testing our classifier model 

# Methods and Analysis 

**Reading in Data**

In [2]:
# Fetch dataset from website
url = "https://github.com/allanji100/dsci-100-group-project/blob/main/heart%2Bdisease/processed.cleveland.data?raw=true"

df = pd.read_csv(
    url,
    names=["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"]
)

#Create a preprocessor with numerical data standardized 
column_transformer = make_column_transformer(
    (StandardScaler(), ["age", "trestbps", "chol", "thalach", "oldpeak"]),
    remainder='passthrough',
    verbose_feature_names_out = False
)

# Drop "ca" and "thal" due to missing values
# Scale numerical features
heart_disease_df = pd.DataFrame(
    column_transformer.fit_transform(df.drop(columns=["ca", "thal"])),
    columns=column_transformer.get_feature_names_out()
)
X = heart_disease_df.drop(columns=["num"])
y = heart_disease_df["num"]

# Create train and test splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123)

In [3]:
heart_disease_df.head()

,age,trestbps,chol,thalach,oldpeak,sex,cp,fbs,restecg,exang,slope,num
0,0.948726,0.757525,-0.264900,0.017197,1.087338,1.0,1.0,1.0,2.0,0.0,3.0,0.0
1,1.392002,1.611220,0.760415,-1.821905,0.397182,1.0,4.0,0.0,2.0,1.0,2.0,2.0
2,1.392002,-0.665300,-0.342283,-0.902354,1.346147,1.0,4.0,0.0,2.0,1.0,2.0,1.0
3,-1.932564,-0.096170,0.063974,1.637359,2.122573,1.0,3.0,0.0,0.0,0.0,3.0,0.0
4,-1.489288,-0.096170,-0.825922,0.980537,0.310912,0.0,2.0,0.0,2.0,0.0,1.0,0.0


***Table 1.*** Sample (top 5 rows) of medical data set from UC Irvine with standardized data and ca and thal collumns removed due to missing data. 

**Visualizing Data**

In [4]:
#filter out categorical data and 
compressed_data_set_summary = df.drop(columns = ["sex", "cp", "fbs", "restecg", "exang", "slope", "ca", "thal", "oldpeak"]).melt(
    id_vars=["age","num"],
    var_name="Parameter",
    value_name="Value",
)

data_summary_chart = alt.Chart(compressed_data_set_summary).mark_bar(clip=True).encode(
    x=alt.X("Value").title("Parameter Value").scale(domain=(0,300)).bin(maxbins=85),
    y=alt.Y("count()").title("Number of Individuals").scale(domain=(0, 15)),
    color= alt.Color("num").title("Heart Disease Severity")
).facet("Parameter", columns = 4)
data_summary_chart


alt.FacetChart(...)

***Figure 1.*** Summarization of cholesterol levels (chol), max heart rate (thalach) and resting blood pressure (trestbps) coloured based on heart disease severity(0 = Healthy, 1-4 = increasing risk of cardiovascular disease). For chol and tresbps, the higher measured values are correlated with a higher proportion of sick to healthy, as shown in the equal distribution of light and dark colouring. For Thalach, we observe the inverse trend where a lower max heart rate correlates with a higher proportion of heart disease severity, highlighting that a higher max heart rate is associated with better cardiovascular health. 


In [5]:
knn = KNeighborsClassifier()
scores = {"name": [], "average_CV_score": []}

# This serves as a baseline to compare the other runs
cv_results = cross_val_score(knn, X_train, y_train)
scores["name"].append("all_features")
scores["average_CV_score"].append(np.average(cv_results))
print(
    "Baseline model with no optimizations: ",
    np.round(np.average(cv_results), decimals=4)
)

# Calculate CV scores with one feature dropped
for col in X_train.columns:
    cv_results = cross_val_score(knn, X_train.drop(columns=[col]), y_train)
    scores["name"].append("no_" + col)
    scores["average_CV_score"].append(np.average(cv_results))

# Convert the dict to a dataframe for visualization purposes
cv_results_df = pd.DataFrame(scores)

# Generate chart with line to show baseline
chart = alt.Chart(cv_results_df).mark_bar(clip=True).encode(
    alt.X(
        'name:O',
        title="CV Run Name"
    ),
    alt.Y(
        'average_CV_score:Q',
        title="Average CV Score"
    ).scale(domain=[0.5, 0.6]),
).properties(
    title = "Average CV Scores w/ Removed Features (Lower is better)"
)
line = alt.Chart(cv_results_df.loc[cv_results_df["name"] == "all_features"]).mark_rule().encode(y='average_CV_score')
CV_results_summary_chart = chart + line

Baseline model with no optimizations:  0.5411


In [6]:
CV_results_summary_chart

alt.LayerChart(...)

***Figure 2.*** Plot of Average CV Scores with Removed Features where a lower score indicates less influence on. This tells us that removing cp, exang, sex, and thalach lowers our overall CV scores. Therefore, these are probably the most useful features. Therefore, we can infer that removing other parameters in our model would be optimal. 

In [7]:
# Drop the unhelpful features
X_train_cropped = X_train.drop(columns=["age", "chol", "fbs", "restecg", "slope", "trestbps"])
X_test_cropped = X_test.drop(columns=["age", "chol", "fbs", "restecg", "slope", "trestbps"])

Now we can compare how our model fairs with and without these cropped features.

In [8]:
# Output the average CV scores
print(
    "With all features: ",
    np.round(
        np.average(cross_val_score(knn, X_train, y_train)),
        decimals=4
    )
)
print(
    "With cropped features: ",
    np.round(
        np.average(cross_val_score(knn, X_train_cropped, y_train)),
        decimals=4
    )
)

With all features:  0.5411
With cropped features:  0.5695


We can see that our model is slightly improved by dropping the bad features.

We can further optimize our model by optimizing our hyper-parameters. In this case, we only need to optimize for n_neighbors in our KNeighborsClassifier.

In [9]:

# Setup and perform grid search to find the best value for K
parameters = {'n_neighbors':range(1, 50)}
grid = GridSearchCV(knn, parameters)

grid.fit(X_train_cropped, y_train)
grid_results = pd.DataFrame(grid.cv_results_)

# Chart the CV scores for each K value
accuracy_vs_k = alt.Chart(grid_results).mark_line(point=True).encode(
    x=alt.X("param_n_neighbors").title("Neighbors"),
    y=alt.Y("mean_test_score")
        .scale(zero=False)
        .title("Accuracy estimate")
).properties(width = 500)



In [10]:
accuracy_vs_k

alt.Chart(...)

***Figure 3.*** Plot of estimated accuracy versus the number of neighbors. We see that the n_neighbours value of 9 achieves the highest accuracy in our grid search. Below, that setting the number of neighbors to 
 36 provides the highest cross-validation accuracy estimate (60.8%)

In [11]:
# Show how the optimized hyper-parameters affect the accuracy of the model
hyper_param_optimized_knn = KNeighborsClassifier(n_neighbors=9)
print(
    "Test score with hyper-parameters optimized: ",
    np.round(
        np.average(cross_val_score(hyper_param_optimized_knn, X_train_cropped, y_train)),
        decimals=4
    )
)

Test score with hyper-parameters optimized:  0.6078


Finally, we can see how our model stacks up against test data:

In [12]:
# Train and show the test accuracy of the final model with all optimizations
final_knn = KNeighborsClassifier(n_neighbors=9).fit(X_train_cropped, y_train)
print(np.round(final_knn.score(X_test_cropped, y_test), decimals=4))

0.5574


We can see that our test score is a bit lower than our cross-validation score. However, we can also see that our optimized model has scored better on the test data, than our unoptmized model did on the training data.

Baseline unoptimized: 0.5411

Optmized: 0.5574

# Discussions 

**Summary**

First, we set the baseline value of average cross validation (CV) scores by using all features in the dataset, which was around 0.54. We then removed one variable at a time and compared the changed CV scores with the baseline and CV scores of other variables. If removal of a feature lowered the scores below the baseline, then we considered the feature was significantly influencing the CV scores. This meant that they would be more useful features to predict for heart disease than other variables. In other words, features that did not change the CV scores were not influencing them, and features that increased the scores were interfering with the accuracy of the model prediction. By this process, we found that cp, exang, sex, and thalach were features that would be most useful for predictions, and dropped all other features that failed to fall below the baseline and analyzed the CV scores.
The CV scores of the model with just the useful features were higher (0.57) than scores of the model with all features (0.54). Since this process was done under the default value of k, we then optimized the k value  for n_neighbors, which we found to be k =9. The CV score increased further when we optimized k and removed non-useful features (0.61). 
After training, we tested our model. We again found that the test score with refined variables is higher (0.56) than the model that wasn’t refined (0.54).


**Expected outcomes**

We expected the refined model to have better prediction scores than the non-refined model since sometimes the variables that are less effective for prediction can interfere with the results. 

However, some variables we found not useful were not reasonably expected. For example, cholesterol is considered to be heavily related to heart disease in both media and literature due to its tendency to clog blood vessels, but our analysis did not find it to be as helpful in predicting heart disease. Another unexpected variable was fasting blood sugar (fbs) levels, since diabetes is also closely related to heart disease we expected this variable to be useful for prediction. However, our analysis did not find fbs to be affecting CV scores.
We also had some variables that we expected to be relevant, such as maximum heart rate (thalach), which were successfully observed in our results. Exercise and fitness are known and proven to decrease the risk of heart disease, which are captured by the significance of the variables thali and exercise-induced angina (Exang). This matches scientific consensus as the paper by Williams (2001) indicates that “the risks of coronary heart disease or cardiovascular disease decrease linearly in association with increasing percentiles of physical activity.” This shows that our model has correctly identified this trend.

It seems like our model was able to capture the relevance of exercise and fitness-related features with heart disease but not for most other health-related features (excluding constrictive pericarditis - cp). There may be some unknown relevance or relation between the features that were unable to be observed by our data, or methods of analysis. 




**Significance of results**

These findings could predict an individual risk of heart disease and allow for preventative measures to be taken beforehand by looking at patterns of relevant variables. This could also direct scholars' attention to looking into more relevant variables to find better ways of predicting heart disease. It could also encourage them to refine the less relevant variables to predict the disease better.


**Future questions**

Our findings could allow scholars to look into why some features are more relevant for heart disease prediction and why some are not. They could also look into the possible interactions and relationship between features and investigate how this improves or worsen prediction. Additionally, the paper by Roeters et al. finds that “no risk factor has been recognized as acting on one gender but not on the other”. Rather, the paper notes that other risk factors may have a higher impact on women instead of men. Our model found that gender has a positive impact on the accuracy of our model. This opens up questions on why just gender seems to have such an impact.

# References

Janosi,Andras, Steinbrunn,William, Pfisterer,Matthias, and Detrano,Robert. (1988). Heart Disease. UCI Machine Learning Repository. https://doi.org/10.24432/C52P4X.

Jeanine E Roeters van Lennep, H.Tineke Westerveld, D.Willem Erkelens, Ernst E van der Wall (2002, February), Risk factors for coronary heart disease: implications of gender, Cardiovascular Research, 53(3), 538–549, https://doi.org/10.1016/S0008-6363(01)00388-1

Williams, P. T. (2001). Physical fitness and activity as separate heart disease risk factors: a meta-analysis. Medicine and Science in Sports and Exercise, 33(5), 754–761. https://doi.org/10.1097/00005768-200105000-00012
